# Import packages

In [24]:
import pandas as pd
import numpy as np

import copy

# Custom function

# Import data

In [25]:
largenet_price_df = pd.read_csv('./Data/Large_network/Demirer_stock_prices.csv')

print(largenet_price_df.head())

print(largenet_price_df.shape)

print(largenet_price_df.tail())



         Date        JPM        BAC          C        WFC          GS  \
0  2010-01-04  32.092087  13.731325  30.160183  20.225887  147.118881   
1  2010-01-05  32.713711  14.177655  31.313356  20.781136  149.719818   
2  2010-01-06  32.893463  14.343938  32.289139  20.810753  148.121780   
3  2010-01-07  33.545033  14.816527  32.377846  21.565891  151.020355   
4  2010-01-08  33.462643  14.685253  31.845606  21.365999  148.164291   

          MS         BK        USB        PNC  ...     D05.SI     U11.SI  \
0  25.570005  22.653957  17.471710  41.086712  ...  10.017633  13.692189   
1  26.504782  22.894192  17.914608  41.501339  ...  10.148582  13.664332   
2  26.843952  22.549856  18.105515  41.501339  ...  10.148582  13.859340   
3  27.232754  23.502777  18.456785  43.751095  ...  10.030727  13.859340   
4  26.678505  23.646927  18.487324  43.866280  ...  10.070014  13.873263   

   EBS.VI  DANSKE.CO        ETE.AT   INGA.AS     DNB.OL  SBER.ME  ISCTR.IS  \
0     NaN  78.602135  2525

In [34]:
# Remove last row as it has lots of missing values

missing_share = sum(largenet_price_df[largenet_price_df.Date == '2020-12-31'].isnull().sum())/\
(largenet_price_df.shape[1]-1) # -1 for Date column

print("Missing share for 2020-12-31: {}".format(round(missing_share,2)))

Missing share for 2020-12-31: 0.94


In [36]:
largenet_price_df = largenet_price_df.drop([2868])

print(largenet_price_df.tail())

            Date         JPM        BAC          C        WFC          GS  \
2863  2020-12-24  123.629646  29.959999  60.570000  29.840000  256.160004   
2864  2020-12-25         NaN        NaN        NaN        NaN         NaN   
2865  2020-12-28  124.443787  30.129999  61.130001  29.930000  259.589996   
2866  2020-12-29  124.116150  30.010000  60.910000  29.780001  258.010010   
2867  2020-12-30  124.463646  29.980000  60.860001  29.750000  259.450012   

             MS         BK        USB         PNC  ...     D05.SI     U11.SI  \
2863  68.089996  41.040001  46.066723  146.149994  ...        NaN        NaN   
2864        NaN        NaN        NaN         NaN  ...        NaN        NaN   
2865  68.050003  41.439999  46.235172  146.539993  ...  25.129999  22.680000   
2866  67.709999  41.439999  45.709999  144.869995  ...  25.200001  22.639999   
2867  67.839996  41.689999  46.150002  146.889999  ...  25.230000  22.750000   

         EBS.VI   DANSKE.CO  ETE.AT  INGA.AS      DNB.OL

# Data preprocessing

## Parameters

In [37]:
start_date = '2012-01-01'
nan_drop_thresh = 0.8
max_gap_allowed = 10

## Filter by Date

In [38]:
largenet_price_df = largenet_price_df[largenet_price_df.Date > start_date]

print(largenet_price_df.shape)

(2348, 87)


## Set Date as index

In [39]:
largenet_price_df = largenet_price_df.set_index('Date')

print(largenet_price_df.head())

                  JPM       BAC          C        WFC         GS         MS  \
Date                                                                          
2012-01-02        NaN       NaN        NaN        NaN        NaN        NaN   
2012-01-03  26.867037  5.115416  25.151731  21.559872  82.778442  13.519601   
2012-01-04  27.037233  5.124233  25.009682  21.658457  82.240227  13.401896   
2012-01-05  27.601957  5.565219  25.311543  22.007292  82.101357  13.687759   
2012-01-06  27.354393  5.450563  25.347055  21.946625  81.094398  13.368262   

                   BK        USB        PNC        COF  ...    D05.SI  \
Date                                                    ...             
2012-01-02        NaN        NaN        NaN        NaN  ...       NaN   
2012-01-03  16.952410  21.653416  46.564701  37.324905  ...  8.118500   
2012-01-04  17.010269  21.645563  46.714569  37.964756  ...  8.229521   
2012-01-05  17.150780  21.967464  47.179993  38.544888  ...  8.187888   
2012-01-

## Log return

In [40]:
cols = largenet_price_df.columns

largenet_return_df = copy.deepcopy(largenet_price_df)

for col in cols:
    largenet_return_df[col] = np.log(largenet_price_df[col]) - np.log(largenet_price_df[col].shift(1))

In [41]:
print("===")
print(largenet_price_df['8331.T'].iloc[:10])

print("===")

print(largenet_return_df['8331.T'].iloc[:10])

===
Date
2012-01-02           NaN
2012-01-03           NaN
2012-01-04    416.442047
2012-01-05    409.049622
2012-01-06    404.942688
2012-01-09           NaN
2012-01-10    405.764038
2012-01-11    408.228180
2012-01-12    405.764038
2012-01-13    402.478485
Name: 8331.T, dtype: float64
===
Date
2012-01-02         NaN
2012-01-03         NaN
2012-01-04         NaN
2012-01-05   -0.017911
2012-01-06   -0.010091
2012-01-09         NaN
2012-01-10         NaN
2012-01-11    0.006054
2012-01-12   -0.006054
2012-01-13   -0.008130
Name: 8331.T, dtype: float64


## Drop columns with all NaN-s

In [42]:
print(largenet_return_df.shape)
largenet_return_df = largenet_return_df.dropna(axis=1, how='all')
largenet_return_df = largenet_return_df.dropna(axis=0, how='all')
print(largenet_return_df.shape)

(2348, 86)
(2347, 86)


In [43]:
thresh = nan_drop_thresh * len(largenet_return_df)
largenet_return_df.dropna(thresh = thresh, axis = 1, inplace = True)

In [44]:
largenet_return_df = largenet_return_df.replace({np.nan: None})

In [45]:
print(largenet_return_df.head())

print(largenet_return_df.shape)

                   JPM         BAC           C         WFC          GS  \
Date                                                                     
2012-01-03        None        None        None        None        None   
2012-01-04  0.00631477  0.00172213 -0.00566369  0.00456219  -0.0065231   
2012-01-05   0.0206718   0.0825555   0.0119975   0.0159779 -0.00169002   
2012-01-06 -0.00900954  -0.0208174  0.00140201 -0.00276048  -0.0123407   
2012-01-09 -0.00169791   0.0144579   0.0183937   0.0123632    0.013503   

                    MS          BK          USB         PNC          COF  ...  \
Date                                                                      ...   
2012-01-03        None        None         None        None         None  ...   
2012-01-04 -0.00874437  0.00340721 -0.000362734  0.00321332    0.0169975  ...   
2012-01-05   0.0211057  0.00822643     0.014762  0.00991384    0.0151652  ...   
2012-01-06  -0.0236185  -0.0145635  -0.00825431  0.00167033 -0.000442699  ..

In [46]:
largenet_return_df.describe()

,JPM,BAC,C,WFC,GS,MS,BK,USB,PNC,COF,...,BCP.LS,D05.SI,U11.SI,EBS.VI,DANSKE.CO,INGA.AS,DNB.OL,SBER.ME,ISCTR.IS,SBK.JO
count,2181.0,2181.0,2181.0,2181.0,2181.0,2181.0,2181.0,2181.0,2181.0,2181.0,...,2262.0,2175.0,2175.0,2044.0,2177.0,2261.0,2194.0,2180.0,2323.0,2279.0
unique,2165.0,2136.0,2172.0,2159.0,2179.0,2164.0,2161.0,2156.0,2167.0,2172.0,...,2049.0,2088.0,2104.0,2006.0,2105.0,2248.0,2134.0,2160.0,2081.0,2207.0
top,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
freq,16.0,44.0,10.0,22.0,3.0,16.0,21.0,23.0,14.0,9.0,...,117.0,76.0,63.0,33.0,58.0,14.0,52.0,20.0,171.0,73.0


In [47]:
print(largenet_return_df.shape)

(2347, 82)


## Gap analysis

In [48]:
n,d = largenet_return_df.shape

In [49]:
cols_to_drop = []

for i in range(0,d):
    ticker = cols[i]
    current_gap = 0
    for j in range(0,n):
        if current_gap == 0:
            if largenet_return_df.iloc[j,i] == None:
                #print("Hello")
                current_gap += 1
                
        else:
            if j > 1:
                if (largenet_return_df.iloc[j,i] == None) & (largenet_return_df.iloc[j-1,i] == None):
                    current_gap += 1

                else:
                    current_gap = 0
                
        if (current_gap > max_gap_allowed) & (ticker not in cols_to_drop):
            #print(ticker)
            cols_to_drop.append(ticker)

In [50]:
print("Number of tickers to drop: {}".format(len(cols_to_drop)))

Number of tickers to drop: 6


In [51]:
# Drop columns with too large gaps
largenet_return_df = largenet_return_df.drop(columns=cols_to_drop)

print(largenet_return_df.shape)

(2347, 76)


## Impute missing return values

In [122]:
df = copy.deepcopy(largenet_return_df) #.iloc[:100,:30]

df = df.replace({np.nan: None})

In [123]:
print(df.shape)

n,d = df.shape

(2347, 76)


In [124]:
print("===")
print(largenet_price_df['8331.T'].iloc[:10])

print("===")

print(df['8331.T'].iloc[:10])

===
Date
2012-01-02           NaN
2012-01-03           NaN
2012-01-04    416.442047
2012-01-05    409.049622
2012-01-06    404.942688
2012-01-09           NaN
2012-01-10    405.764038
2012-01-11    408.228180
2012-01-12    405.764038
2012-01-13    402.478485
Name: 8331.T, dtype: float64
===
Date
2012-01-03         NaN
2012-01-04         NaN
2012-01-05   -0.017911
2012-01-06   -0.010091
2012-01-09         NaN
2012-01-10         NaN
2012-01-11    0.006054
2012-01-12   -0.006054
2012-01-13   -0.008130
2012-01-16   -0.008197
Name: 8331.T, dtype: float64


In [125]:
# Check whether there are any missing values
df.isna().sum()

JPM         166
BAC         166
C           166
WFC         166
GS          166
           ... 
INGA.AS      86
DNB.OL      153
SBER.ME     167
ISCTR.IS     24
SBK.JO       68
Length: 76, dtype: int64

In [126]:
display(df)

,JPM,BAC,C,WFC,GS,MS,BK,USB,PNC,COF,...,BANKBARODA.NS,D05.SI,U11.SI,EBS.VI,DANSKE.CO,INGA.AS,DNB.OL,SBER.ME,ISCTR.IS,SBK.JO
Date,,,,,,,,,,,,,,,,,,,,,
2012-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.046850,NaN,NaN,NaN,0.015605,0.012708,-0.011945,NaN,0.060977,0.016671
2012-01-04,0.006315,0.001722,-0.005664,0.004562,-0.006523,-0.008744,0.003407,-0.000363,0.003213,0.016997,...,0.001142,0.013582,0.010873,NaN,-0.027471,-0.014286,-0.013829,NaN,-0.046117,0.005859
2012-01-05,0.020672,0.082555,0.011998,0.015978,-0.001690,0.021106,0.008226,0.014762,0.009914,0.015165,...,-0.009243,-0.005072,-0.005742,NaN,-0.005319,-0.035542,0.006938,NaN,-0.008889,-0.010951
2012-01-06,-0.009010,-0.020817,0.001402,-0.002760,-0.012341,-0.023619,-0.014563,-0.008254,0.001670,-0.000443,...,0.010953,0.005072,-0.014176,NaN,-0.011398,-0.008215,-0.003463,NaN,-0.018019,0.007877
2012-01-09,-0.001698,0.014458,0.018394,0.012363,0.013503,0.018692,0.020809,0.017504,0.008146,0.023850,...,-0.009155,-0.008468,-0.016356,NaN,0.004709,0.006395,-0.002605,NaN,-0.030772,-0.006876
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-24,-0.004407,-0.003000,-0.003461,-0.016947,-0.001131,-0.001761,0.001463,-0.005149,-0.003756,0.004945,...,0.004942,NaN,NaN,NaN,NaN,0.008117,NaN,-0.000818,-0.002981,-0.002105
2020-12-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.006858,0.005952,NaN
2020-12-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.014049,0.013265,NaN


In [127]:
for i in range(0,d):
    #print("COL")
    #print(i)
    
    j = 0
    while j < n:
        
        #print("ROW")
        #print(j)
        
        if j == 0:
            # Impute overall average return if first value is missing
            k = j
            while (np.isnan(df.iloc[k,i])) & (k < n-1):
                k += 1
                #print("INNER ROW")
                #print(k)
                
                
            if k > j:
                
                df.iloc[j:k,i] = np.nanmean(df.iloc[:,i].astype(float))
                
                # Increment the counter
                j = k
                
            else:
                
                j += 1
        
        else:
            k = j
            while np.isnan(df.iloc[k,i]) & (k < n-1):
                k += 1
                
            if k > j:
            
                # Get dates with the valid price data
                start_date = df.index[j-1]
                end_date = df.index[k]

                #print(start_date)
                #print(end_date)

                # Get valid prices
                start_price = largenet_price_df.loc[start_date][cols[i]]
                end_price = largenet_price_df.loc[end_date][cols[i]]

                # Compute daily log returns
                daily_return = np.log(end_price/start_price) / (k-j)
                if np.isnan(daily_return):
                    daily_return = np.nanmean(df.iloc[:,i].astype(float))
                    #print(daily_return)
                    #print("problem at {}, date {}".format(cols[i],largenet_price_df.index[j]))

                # Impute the computed values
                df.iloc[j:k,i] = daily_return

                # Increment the counter
                j = k
                
            else:
                j += 1

In [128]:
display(df)

,JPM,BAC,C,WFC,GS,MS,BK,USB,PNC,COF,...,BANKBARODA.NS,D05.SI,U11.SI,EBS.VI,DANSKE.CO,INGA.AS,DNB.OL,SBER.ME,ISCTR.IS,SBK.JO
Date,,,,,,,,,,,,,,,,,,,,,
2012-01-03,0.000733,0.000783,0.000409,0.000168,0.000490,0.000754,0.000398,0.000385,0.000570,0.000502,...,0.046850,0.000559,0.000369,0.000359,0.015605,0.012708,-0.011945,0.000778,0.060977,0.016671
2012-01-04,0.006315,0.001722,-0.005664,0.004562,-0.006523,-0.008744,0.003407,-0.000363,0.003213,0.016997,...,0.001142,0.013582,0.010873,0.000359,-0.027471,-0.014286,-0.013829,0.000778,-0.046117,0.005859
2012-01-05,0.020672,0.082555,0.011998,0.015978,-0.001690,0.021106,0.008226,0.014762,0.009914,0.015165,...,-0.009243,-0.005072,-0.005742,0.000359,-0.005319,-0.035542,0.006938,0.000778,-0.008889,-0.010951
2012-01-06,-0.009010,-0.020817,0.001402,-0.002760,-0.012341,-0.023619,-0.014563,-0.008254,0.001670,-0.000443,...,0.010953,0.005072,-0.014176,0.000359,-0.011398,-0.008215,-0.003463,0.000778,-0.018019,0.007877
2012-01-09,-0.001698,0.014458,0.018394,0.012363,0.013503,0.018692,0.020809,0.017504,0.008146,0.023850,...,-0.009155,-0.008468,-0.016356,0.000359,0.004709,0.006395,-0.002605,0.000778,-0.030772,-0.006876
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-24,-0.004407,-0.003000,-0.003461,-0.016947,-0.001131,-0.001761,0.001463,-0.005149,-0.003756,0.004945,...,0.004942,0.002690,-0.001450,-0.005137,-0.005071,0.008117,0.009765,-0.000818,-0.002981,-0.002105
2020-12-25,0.001964,0.000834,0.002799,-0.001006,0.003598,-0.002798,0.004850,-0.003887,-0.004398,-0.005789,...,0.000110,0.002690,-0.001450,-0.005137,-0.005071,0.020287,0.009765,0.006858,0.005952,-0.004455
2020-12-28,0.001964,0.000834,0.002799,-0.001006,0.003598,-0.002798,0.004850,-0.003887,-0.004398,-0.005789,...,0.000110,0.002690,-0.001450,-0.005137,-0.005071,0.020287,0.009765,0.014049,0.013265,-0.004455


In [129]:
sum(df.iloc[0:n,:].isna().sum())

3

In [130]:
df.iloc[-1,:].isna().sum()

3

In [131]:
# Since all of the missing values are in the last column, just impute them with the column mean
df = df.apply(lambda x: x.fillna(x.mean()),axis=0)
#df=df.fillna(df.mean())

In [133]:
# Check whether there are any missing values
assert(sum(df.isna().sum())==0)

In [134]:
display(df)

,JPM,BAC,C,WFC,GS,MS,BK,USB,PNC,COF,...,BANKBARODA.NS,D05.SI,U11.SI,EBS.VI,DANSKE.CO,INGA.AS,DNB.OL,SBER.ME,ISCTR.IS,SBK.JO
Date,,,,,,,,,,,,,,,,,,,,,
2012-01-03,0.000733,0.000783,0.000409,0.000168,0.000490,0.000754,0.000398,0.000385,0.000570,0.000502,...,0.046850,0.000559,0.000369,0.000359,0.015605,0.012708,-0.011945,0.000778,0.060977,0.016671
2012-01-04,0.006315,0.001722,-0.005664,0.004562,-0.006523,-0.008744,0.003407,-0.000363,0.003213,0.016997,...,0.001142,0.013582,0.010873,0.000359,-0.027471,-0.014286,-0.013829,0.000778,-0.046117,0.005859
2012-01-05,0.020672,0.082555,0.011998,0.015978,-0.001690,0.021106,0.008226,0.014762,0.009914,0.015165,...,-0.009243,-0.005072,-0.005742,0.000359,-0.005319,-0.035542,0.006938,0.000778,-0.008889,-0.010951
2012-01-06,-0.009010,-0.020817,0.001402,-0.002760,-0.012341,-0.023619,-0.014563,-0.008254,0.001670,-0.000443,...,0.010953,0.005072,-0.014176,0.000359,-0.011398,-0.008215,-0.003463,0.000778,-0.018019,0.007877
2012-01-09,-0.001698,0.014458,0.018394,0.012363,0.013503,0.018692,0.020809,0.017504,0.008146,0.023850,...,-0.009155,-0.008468,-0.016356,0.000359,0.004709,0.006395,-0.002605,0.000778,-0.030772,-0.006876
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-24,-0.004407,-0.003000,-0.003461,-0.016947,-0.001131,-0.001761,0.001463,-0.005149,-0.003756,0.004945,...,0.004942,0.002690,-0.001450,-0.005137,-0.005071,0.008117,0.009765,-0.000818,-0.002981,-0.002105
2020-12-25,0.001964,0.000834,0.002799,-0.001006,0.003598,-0.002798,0.004850,-0.003887,-0.004398,-0.005789,...,0.000110,0.002690,-0.001450,-0.005137,-0.005071,0.020287,0.009765,0.006858,0.005952,-0.004455
2020-12-28,0.001964,0.000834,0.002799,-0.001006,0.003598,-0.002798,0.004850,-0.003887,-0.004398,-0.005789,...,0.000110,0.002690,-0.001450,-0.005137,-0.005071,0.020287,0.009765,0.014049,0.013265,-0.004455


In [135]:
print(df.shape)

(2347, 76)


# Export final dataset

In [136]:
df.to_csv('./Data/Large_network/largenet_log_ret.csv')